In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

print(f"TensorFlow version: {tf.__version__}")

2026-02-24 13:12:35.159511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.18.0


In [3]:
# Paths
processed_dir = Path("../data/processed")
splits_dir = Path("../data/splits")
models_dir = Path("../models")
models_dir.mkdir(parents=True, exist_ok=True)

# Load preprocessing config
with open(processed_dir / "preprocessing_config.json") as f:
    config = json.load(f)

# Load class mappings
with open(processed_dir / "class_to_index.json") as f:
    class_to_index = json.load(f)

with open(processed_dir / "index_to_class.json") as f:
    index_to_class = {int(k): v for k, v in json.load(f).items()}

# Load class weights
with open(processed_dir / "class_weights.json") as f:
    class_weights = {int(k): v for k, v in json.load(f).items()}

# Core config
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
N_CLASSES = len(class_to_index)
MEAN = config['normalization']['mean']
STD = config['normalization']['std']
RANDOM_SEED = config['random_seed']

print(f"Classes: {N_CLASSES}")
print(f"Image size: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Normalization mean: {MEAN}")
print(f"Normalization std: {STD}")

Classes: 15
Image size: (224, 224)
Batch size: 32
Normalization mean: [0.46, 0.48, 0.42]
Normalization std: [0.21, 0.18, 0.22]


In [4]:
# Load split manifests
train_df = pd.read_csv(splits_dir / "train.csv")
val_df = pd.read_csv(splits_dir / "val.csv")
test_df = pd.read_csv(splits_dir / "test.csv")

print(f"Train: {len(train_df)} images")
print(f"Val:   {len(val_df)} images")
print(f"Test:  {len(test_df)} images")

Train: 14227 images
Val:   3049 images
Test:  3049 images


In [5]:
# Normalization
mean = tf.constant(MEAN, dtype=tf.float32)
std = tf.constant(STD, dtype=tf.float32)

def normalize(image):
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - mean) / std
    return image

# Minority classes (< 500 images) get aggressive augmentation
MINORITY_CLASSES = [
    cls for cls, idx in class_to_index.items()
    if len(train_df[train_df['class_name'] == cls]) < 500
]
print(f"Minority classes (aggressive augmentation): {MINORITY_CLASSES}")

def augment_standard(image):
    """Standard augmentation for majority classes"""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image

def augment_aggressive(image):
    """Aggressive augmentation for minority classes"""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.4)
    image = tf.image.random_contrast(image, lower=0.6, upper=1.4)
    image = tf.image.random_saturation(image, lower=0.6, upper=1.4)
    image = tf.image.random_hue(image, max_delta=0.1)
    return image

def augment_denoise(image):
    """Denoising augmentation weighted toward Tomato classes"""
    image = augment_standard(image)
    # Light Gaussian smoothing to improve robustness to noise
    image = tf.expand_dims(image, 0)
    image = tf.squeeze(
        tf.nn.avg_pool2d(image, ksize=3, strides=1, padding='SAME'), 0
    )
    return image

Minority classes (aggressive augmentation): ['Potato___healthy', 'Tomato__Tomato_mosaic_virus']


In [7]:
TOMATO_NOISY_CLASSES = [
    'Tomato_healthy',
    'Tomato__Target_Spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite'
]

MINORITY_CLASSES = [
    cls for cls, idx in class_to_index.items()
    if len(train_df[train_df['class_name'] == cls]) < 500
]
print(f"Minority classes (aggressive augmentation): {MINORITY_CLASSES}")

def load_and_preprocess(image_path, label, class_name, is_training=False):
    # Load image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)

    if is_training:
        # Decode class_name tensor to Python string for comparison
        cls = class_name.numpy().decode('utf-8')
        if cls in MINORITY_CLASSES:
            image = augment_aggressive(image)
        elif cls in TOMATO_NOISY_CLASSES:
            image = augment_denoise(image)
        else:
            image = augment_standard(image)

    image = normalize(image)
    return image, label

def build_dataset(df, is_training=False, shuffle=False):
    paths = df['image_path'].values
    labels = df['class_index'].values
    class_names = df['class_name'].values

    dataset = tf.data.Dataset.from_tensor_slices((paths, labels, class_names))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df), seed=RANDOM_SEED)

    if is_training:
        # Use py_function to allow Python string operations inside graph mode
        dataset = dataset.map(
            lambda path, label, cls: tf.py_function(
                func=lambda p, l, c: load_and_preprocess(p, l, c, is_training=True),
                inp=[path, label, cls],
                Tout=(tf.float32, tf.int64)
            ),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    else:
        dataset = dataset.map(
            lambda path, label, cls: load_and_preprocess(path, label, cls, is_training=False),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    # Restore shape info lost by py_function
    dataset = dataset.map(
        lambda img, label: (tf.ensure_shape(img, [*IMAGE_SIZE, 3]), label)
    )

    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = build_dataset(train_df, is_training=True, shuffle=True)
val_dataset = build_dataset(val_df, is_training=False, shuffle=False)
test_dataset = build_dataset(test_df, is_training=False, shuffle=False)

print("Datasets built successfully")

# Verify batch shape
for images, labels in train_dataset.take(1):
    print(f"Batch image shape: {images.shape}")
    print(f"Batch label shape: {labels.shape}")

Minority classes (aggressive augmentation): ['Potato___healthy', 'Tomato__Tomato_mosaic_virus']
Datasets built successfully


2026-02-24 13:21:31.676147: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


Batch image shape: (32, 224, 224, 3)
Batch label shape: (32,)


2026-02-24 13:21:32.001120: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-24 13:21:32.009048: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 19267840 bytes after encountering the first element of size 19267840 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2026-02-24 13:21:32.039868: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
def build_cnn(input_shape=(224, 224, 3), n_classes=15):
    inputs = keras.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Block 4
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Classifier head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

model = build_cnn(input_shape=(224, 224, 3), n_classes=N_CLASSES)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 1,317,423 (5.03 MB)

 Trainable params: 1,314,479 (5.01 MB)

 Non-trainable params: 2,944 (11.50 KB)

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
training_callbacks = [
    # Stop early if val_loss stops improving
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    # Save best model
    callbacks.ModelCheckpoint(
        filepath=str(models_dir / 'best_model.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    # Reduce LR when val_loss plateaus
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    # TensorBoard logs
    callbacks.TensorBoard(
        log_dir='../logs',
        histogram_freq=1
    )
]

In [12]:
TOMATO_NOISY_CLASSES = [
    'Tomato_healthy',
    'Tomato__Target_Spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite'
]

MINORITY_CLASSES = [
    cls for cls, idx in class_to_index.items()
    if len(train_df[train_df['class_name'] == cls]) < 500
]
print(f"Minority classes (aggressive augmentation): {MINORITY_CLASSES}")

# Get class indices for each augmentation group
minority_indices = [class_to_index[cls] for cls in MINORITY_CLASSES]
noisy_indices = [class_to_index[cls] for cls in TOMATO_NOISY_CLASSES if cls in class_to_index]

def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = normalize(image)
    return image, label

def apply_augmentation(image, label):
    """Apply augmentation based on class index using tf.cond"""
    is_minority = tf.reduce_any(tf.equal(label, tf.constant(minority_indices, dtype=tf.int64)))
    is_noisy = tf.reduce_any(tf.equal(label, tf.constant(noisy_indices, dtype=tf.int64)))

    image = tf.cond(
        is_minority,
        lambda: augment_aggressive(image),
        lambda: tf.cond(
            is_noisy,
            lambda: augment_denoise(image),
            lambda: augment_standard(image)
        )
    )
    return image, label

def build_dataset(df, is_training=False, shuffle=False):
    paths = df['image_path'].values
    labels = df['class_index'].values.astype(np.int64)

    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df), seed=RANDOM_SEED)

    dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if is_training:
        dataset = dataset.map(apply_augmentation, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = build_dataset(train_df, is_training=True, shuffle=True)
val_dataset = build_dataset(val_df, is_training=False, shuffle=False)
test_dataset = build_dataset(test_df, is_training=False, shuffle=False)

print("Datasets built successfully")

# Verify batch shape
for images, labels in train_dataset.take(1):
    print(f"Batch image shape: {images.shape}")
    print(f"Batch label shape: {labels.shape}")
    print(f"Label dtype: {labels.dtype}")

Minority classes (aggressive augmentation): ['Potato___healthy', 'Tomato__Tomato_mosaic_virus']
Datasets built successfully
Batch image shape: (32, 224, 224, 3)
Batch label shape: (32,)
Label dtype: <dtype: 'int64'>


2026-02-24 14:48:22.328470: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-24 14:48:22.423546: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-24 14:48:22.440242: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
EPOCHS = 100

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=training_callbacks,
    verbose=1
)

print("Training complete.")

Epoch 1/100
 52/445 ━━━━━━━━━━━━━━━━━━━━ 1:08:56 11s/step - accuracy: 0.2648 - loss: 2.4933

KeyboardInterrupt: 